# Systems genetics 2020
## Final Project  

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Load data files

In [2]:
liver_exp = pd.read_csv('data/GSE17522_series_matrix_liver.txt', sep = "\t")
brain_exp = pd.read_csv('data/GSE36674_series_matrix_hypothalamus.txt', sep = "\t")

liver_annotations = pd.read_csv('data/annotations_liver_GPL6466-9752.txt', sep = "\t")
brain_annotations = pd.read_csv('data/annotation_brain.annot', sep = "\t")

genotypes = pd.read_excel('data/genotypes.xls', headers=None)
phenotypes = pd.read_excel('data/phenotypes.xls')

mgi = pd.read_csv('data/MGI_Coordinates.Build37.rpt.txt', sep = "\t", error_bad_lines=False, warn_bad_lines=False)

### 2. Gene expression data preprocessing

#### • Make sure that the data is normalized

In [3]:
liver_exp[liver_exp['!Sample_title'] == '!Sample_data_processing']['Liver_C57BL6J_M_B1_rep1'].iloc[0]

'Data were extracted from the scanned image using Agilent Feature Extraction software version 6.1. A total of 122 arrays were run in 8 batches. The samples were semirandomly distributed throughout the batches prior to microarray analysis in order to separate sexes and strains, and to minimize between- and within- batch bias. Technical and biological replicates were run both within each batch and between batches. The microarray data was deposited in the UNC Microarray Database and extracted using Log2 ratios of the mean red channel intensity over the mean green channel intensity. This was followed by LOWESS normalization to remove the intensity dependent dye bias3. Neither the genes nor the arrays were centered. Inter-batch normalization was carried out using a nested ANOVA mixed model with samples within each batch crossed with sex and strain.'

In [4]:
brain_exp[brain_exp['!Sample_title'] == '!Sample_data_processing']['BXD44_F'].iloc[0]

'Probe intensity values were extracted using the Affymetrix GeneChip Operating Software and RMA normalized'

#### • Merge data file with annotation file to get your input matrix

In [5]:
def data_annotations_merge(df_exp, df_annotations):
    """ Merge gene expression dataframe with annotations file """
    
    # Remove metadata rows to get the raw data only
    df_exp = df_exp[~df_exp['!Sample_title'].str.contains('!Sample_', na=False)]
    df_exp = df_exp[~df_exp['!Sample_title'].str.contains('!series_', na=False)]
    df_exp = df_exp[df_exp['!Sample_title'] != 'ID_REF']

    # Rename ID column to match the annotation matrix
    df_exp = df_exp.rename(columns = {'!Sample_title' : 'ID'})

    # Merge with annotation matrix to get the gene identifier (gene symbol)
    df_annotations = df_annotations[['ID', 'GENE_SYMBOL']]
    input_matrix = df_annotations.merge(df_exp, left_on='ID', right_on='ID')
    
    return input_matrix  

Brain

In [6]:
brain_annotations = brain_annotations.rename(columns={'Gene symbol' : 'GENE_SYMBOL'})
brain_matrix = data_annotations_merge(brain_exp, brain_annotations)

Liver

In [7]:
liver_matrix = data_annotations_merge(liver_exp, liver_annotations)

# Keep BXD columns and identifier columns only
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)
id_cols = ['ID', 'GENE_SYMBOL']
liver_matrix = liver_matrix[id_cols + liver_bxd_cols]

# Rename BXD columns
liver_matrix = liver_matrix.rename(columns={col : col.split('_')[1] + '_' + col.split('_')[2] \
                                            for col in liver_matrix.columns.drop(id_cols)})

#### • Remove rows with no gene identifier, <br> • Remove rows with low maximal value.  <br> • Remove rows with low variance.  <br> • Average multiple rows


In [8]:
max_treshold = 0.7
var_treshold = 0.2

# Remove rows with no gene identifier
liver_matrix = liver_matrix[~liver_matrix['GENE_SYMBOL'].isna()]
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)

for col in liver_bxd_cols:
    liver_matrix[col] = liver_matrix[col].astype('float64')

# Filter by maximal value
liver_matrix['max'] = liver_matrix.drop(columns=['ID', 'GENE_SYMBOL']).max(axis=1)
liver_matrix = liver_matrix[liver_matrix['max'] >= max_treshold]
liver_matrix = liver_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(liver_matrix)))

# Filter by variance
liver_matrix['var'] = liver_matrix.drop(columns=['ID', 'GENE_SYMBOL']).var(axis=1)
liver_matrix = liver_matrix[liver_matrix['var'] >= var_treshold]
liver_matrix = liver_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(liver_matrix)))

# Group multiple rows by mean
liver_matrix = liver_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(liver_matrix))

Num of rows after removing all rows with maximal values less than 0.70: 7790 
Num of rows after removing all rows with variance less than 0.20: 1310 
Num of rows after removing duplicated rows: %d  1288


In [9]:
max_treshold = 9
var_treshold = 0.03

# Remove rows with no gene identifier
brain_matrix = brain_matrix[~brain_matrix['GENE_SYMBOL'].isna()]
brain_bxd_cols = list(brain_matrix.filter(regex=("BXD*")).columns)

for col in brain_bxd_cols:
    brain_matrix[col] = brain_matrix[col].astype('float64')

# Filter by maximal value
brain_matrix['max'] = brain_matrix.drop(columns=['ID', 'GENE_SYMBOL']).max(axis=1)
brain_matrix = brain_matrix[brain_matrix['max'] >= max_treshold]
brain_matrix = brain_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(brain_matrix)))

# Filter by variance
brain_matrix['var'] = brain_matrix.drop(columns=['ID', 'GENE_SYMBOL']).var(axis=1)
brain_matrix = brain_matrix[brain_matrix['var'] >= var_treshold]
brain_matrix = brain_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(brain_matrix)))

# Group multiple rows by mean
brain_matrix = brain_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(brain_matrix))

Num of rows after removing all rows with maximal values less than 9.00: 9899 
Num of rows after removing all rows with variance less than 0.03: 2069 
Num of rows after removing duplicated rows: %d  1792


### 3. eQTL analysis